In [24]:
import numpy as np
import pandas as pd

In [25]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [26]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA3a/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [27]:
mineralogy = mineralogy.dropna()

In [28]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [29]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [30]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [31]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
26935,72.456370,0.292183,14.583615,0.657742,1.408910,0.042525,0.626864,2.153932,3.373165,4.404694
26936,72.457581,0.292140,14.583131,0.657654,1.408651,0.042519,0.626681,2.153555,3.373216,4.404871
26937,72.458767,0.292098,14.582657,0.657568,1.408397,0.042513,0.626501,2.153186,3.373267,4.405046
26938,72.459928,0.292057,14.582192,0.657484,1.408148,0.042507,0.626325,2.152825,3.373316,4.405217
27719,72.452496,0.292319,14.585159,0.658038,1.409736,0.042545,0.627456,2.155132,3.373000,4.404120


In [32]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
26935,72.456370,0.292183,14.583615,0.657742,1.408910,0.042525,0.626864,2.153932,3.373165,4.404694
26936,72.457581,0.292140,14.583131,0.657654,1.408651,0.042519,0.626681,2.153555,3.373216,4.404871
26937,72.458767,0.292098,14.582657,0.657568,1.408397,0.042513,0.626501,2.153186,3.373267,4.405046
26938,72.459928,0.292057,14.582192,0.657484,1.408148,0.042507,0.626325,2.152825,3.373316,4.405217
27719,72.452496,0.292319,14.585159,0.658038,1.409736,0.042545,0.627456,2.155132,3.373000,4.404120
...,...,...,...,...,...,...,...,...,...,...
331197,65.219820,0.544563,16.415014,1.172837,3.114096,0.079018,2.264693,4.570141,3.056038,3.563779
331198,65.218588,0.544606,16.415229,1.172794,3.114463,0.079026,2.264999,4.570635,3.055986,3.563675
331199,65.217342,0.544648,16.415446,1.172752,3.114834,0.079034,2.265309,4.571133,3.055934,3.563569
331983,65.221651,0.544501,16.414691,1.172911,3.113547,0.079006,2.264243,4.569404,3.056114,3.563934


In [33]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:135000]

In [35]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA3a/area3a_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA3a/area3a_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA3a/area3a_3.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [37]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3a/area3a_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3a/area3a_2_results_excel.txt")
df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3a/area3a_3_results_excel.txt")

In [39]:
df_final = pd.concat([df_final1, df_final2, df_final3])

In [40]:
df_final

,Q,P,K
SAMPLE,,,
26935,29.678824,36.371857,28.881962
26936,29.680275,36.370402,28.883070
26937,29.681707,36.368947,28.884175
26938,29.683092,36.367545,28.885223
27719,29.674175,36.376515,28.878452
...,...,...,...
331197,19.272211,43.818407,23.644591
331198,19.270172,43.819178,23.643898
331199,19.268105,43.819881,23.643256


## QAPF classification

In [41]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [42]:
df_final["QAPF"].unique()

array(['monzo granite', 'quartz monzodiorite\nquartz monzogabbro',
       'granodiorite'], dtype=object)

In [43]:
df_final.to_excel("../_CIPW/CIPW/AREA3a/qapf.xlsx", index=True)

-------

In [44]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA3a/QAPF_counts.xlsx")

-----

In [45]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area3_a.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [46]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
15,598837.908412,7.715890e+06,55.0,W,8.0,41.948841,25.957568,29.502100,monzo granite
119,635439.679361,7.725109e+06,55.0,W,8.0,39.237976,26.709324,30.287969,monzo granite
400,592263.140650,7.717475e+06,55.0,W,8.0,36.942165,28.927268,30.441601,monzo granite
888,603288.106669,7.703025e+06,55.0,W,8.0,33.246529,29.794734,34.120800,monzo granite
1305,636193.302213,7.723289e+06,55.0,W,8.0,34.608631,29.864238,29.298348,monzo granite
1511,598296.065461,7.697260e+06,55.0,W,8.0,31.413958,28.903626,37.600814,monzo granite
1650,637385.481435,7.725232e+06,55.0,W,8.0,31.570049,38.081322,24.482859,monzo granite
1905,591461.767224,7.704428e+06,55.0,W,8.0,27.089623,36.907690,32.173481,monzo granite
1942,585316.079410,7.693012e+06,55.0,W,8.0,27.494003,45.085424,20.828580,granodiorite
2139,636637.553783,7.727073e+06,55.0,W,8.0,28.614622,38.556488,24.188587,monzo granite


In [47]:
QAPF_control.to_excel("../_CIPW/CIPW/AREA3a/QAPF_control_points.xlsx", index=True)